<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/air_pollution_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

First, I used the data about air_polution data from 
Ministry of Environment

In [ ]:
import pandas as pd
import json
import tensorflow as tf
import tensorflow.keras as K
import numpy as np

I got the tip from how to read the file with utf-8 encoding from [here](https://teddylee777.github.io/pandas/%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%95%9C%EA%B8%80%EA%B9%A8%EC%A7%90%ED%98%84%EC%83%81-%ED%95%B4%EA%B2%B0%EB%B0%A9%EB%B2%95)

In [ ]:
def read_data():
  data = pd.read_csv("/content/drive/My Drive/Datasets/air_2019.csv", encoding='utf-8')
  return data

In [ ]:
data = read_data()

In [ ]:
data

,측정일자,권역코드,권역명,측정소코드,측정소명,미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),초미세먼지(㎍/㎥)
0,20191231,100,도심권,111121,중구,26,18,0.023,0.012,0.4,0.002
1,20191231,100,도심권,111123,종로구,27,19,0.021,0.015,0.4,0.003
2,20191231,100,도심권,111131,용산구,14,15,0.020,0.015,0.4,0.003
3,20191231,101,서북권,111181,은평구,24,13,0.024,0.010,0.4,0.003
4,20191231,101,서북권,111191,서대문구,23,9,0.023,0.013,0.5,0.002
...,...,...,...,...,...,...,...,...,...,...,...
8891,20190101,103,서남권,111301,양천구,46,29,0.014,0.033,0.6,0.002
8892,20190101,104,동남권,111261,강남구,37,31,0.012,0.028,0.6,0.005
8893,20190101,104,동남권,111262,서초구,37,20,0.015,0.032,0.4,0.004
8894,20190101,104,동남권,111273,송파구,44,28,0.011,0.042,0.8,0.004


Filtering data

In [ ]:
location = data['측정소명'] == "중구"

dust_location = data[location]
new_list = pd.DataFrame(dust_location['미세먼지(㎍/㎥)'])
new_list

,미세먼지(㎍/㎥)
0,26
25,36
50,46
75,43
100,34
...,...
8771,63
8796,55
8821,35
8846,31


In [ ]:
def read_dust_data():
  data = pd.read_excel("/content/drive/My Drive/Datasets/APIS/APIS_data.xlsx", encoding='utf-8')
  return data

In [ ]:
data = read_dust_data()

In [ ]:
data

,1.0ug/m3,2.5ug/m3
0,21,29
1,21,30
2,21,30
3,22,32
4,22,32
...,...,...
3327,25,37
3328,25,36
3329,26,37
3330,26,38


# 공공데이터 받기

In [8]:
import urllib
from urllib.parse import urlencode, quote_plus, unquote
from urllib.request import urlopen
import requests
import json


DECODE_KEY = unquote('cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D')


In [9]:
def api_Living_Reading(key, feature, time):
  #-*- coding:utf-8 -*-
  url = f"http://apis.data.go.kr/1360000/LivingWthrIdxService/{feature}"
  
  queryParams = '?' + urlencode({ quote_plus('ServiceKey') : key, quote_plus('pageNo') : '1', quote_plus('dataType') : 'JSON', quote_plus('areaNo') : '1100000000', quote_plus('time') : time,quote_plus('requestCode') : 'A20'})
  request = requests.get(url + queryParams)

  json_data = json.loads(request.text)
  return json_data["response"]['body']["items"]["item"][0]

In [44]:
items = api_Living_Reading(DECODE_KEY, 'getHeatFeelingIdx', '2020072706')

In [45]:
heatfeeling = items['h3'] # 더위체감지수
heatfeeling

'23'

In [46]:
items = api_Living_Reading(DECODE_KEY, 'getDiscomfortIdx', '2020072706')

In [47]:
discomfort = items['h3'] # 불쾌지수
discomfort

'73'

In [55]:
items = api_Living_Reading(DECODE_KEY, 'getUVIdx', '2020072723')

In [56]:
uv = items['today'] # 자외선지수
uv

''

In [53]:
items = api_Living_Reading(DECODE_KEY, 'getSenTaIdx', '2020072723')

In [54]:
heat = items['h3'] # 체감온도
heat

'23'

## Time Module

In [1]:
import time

now = time.localtime()
now

time.struct_time(tm_year=2020, tm_mon=7, tm_mday=28, tm_hour=14, tm_min=10, tm_sec=47, tm_wday=1, tm_yday=210, tm_isdst=0)

In [6]:
DATE = f"{now.tm_year}{now.tm_mon:02d}{now.tm_mday:02d}{now.tm_hour:02d}"

In [5]:
DATE

'2020072814'

In [10]:
items = api_Living_Reading(DECODE_KEY, 'getDiscomfortIdx', DATE)

In [11]:
items

{'areaNo': '1100000000',
 'code': 'A06',
 'date': '2020072812',
 'h12': '74',
 'h15': '74',
 'h18': '74',
 'h21': '76',
 'h24': '77',
 'h27': '77',
 'h3': '77',
 'h30': '79',
 'h33': '75',
 'h36': '76',
 'h39': '74',
 'h42': '74',
 'h45': '75',
 'h48': '76',
 'h51': '77',
 'h54': '78',
 'h57': '76',
 'h6': '78',
 'h60': '74',
 'h63': '',
 'h66': '',
 'h9': '75'}

In [18]:
items = api_Living_Reading(DECODE_KEY, 'getAirDiffusionIdx', DATE)

In [20]:
items['h3'] # 대기확산지수

'50'

## If time is same...

In [14]:
lasttime = DATE

In [15]:
if lasttime != DATE:
  lasttime = DATE

## overall function

In [17]:
import urllib
from urllib.parse import urlencode, quote_plus, unquote
from urllib.request import urlopen
import requests
import json
import time

In [ ]:
now = time.localtime()

In [40]:
DECODE_KEY = unquote('cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D')
DATE = f"{now.tm_year}{now.tm_mon:02d}{now.tm_mday:02d}{now.tm_hour:02d}"
QUERY = ['getHeatFeelingIdx','getDiscomfortIdx','getUVIdx','getSenTaIdx','getAirDiffusionIdx']
lasttime = ""

In [26]:
def api_Living_Reading(key, feature, time, areano='1100000000'):
  #-*- coding:utf-8 -*-
  url = f"http://apis.data.go.kr/1360000/LivingWthrIdxService/{feature}"
  
  queryParams = '?' + urlencode({ quote_plus('ServiceKey') : key, quote_plus('pageNo') : '1', quote_plus('dataType') : 'JSON', quote_plus('areaNo') : areano, quote_plus('time') : time, quote_plus('requestCode') : 'A20'})
  request = requests.get(url + queryParams)
  json_data = json.loads(request.text)

  if feature == 'getUVIdx':
    return json_data["response"]['body']["items"]["item"][0]['today']
  else:
    return json_data["response"]['body']["items"]["item"][0]['h3']

In [62]:
def get_living_weather_data(key, query, time, lasttime,result, areano='1100000000'):
  if lasttime == DATE:
    return result, lasttime
  else:
    lasttime = DATE
    print("Receiving now data")
    for feature in QUERY:
      result[feature] = api_Living_Reading(DECODE_KEY, feature, DATE)
    return result, lasttime

In [63]:
result, lasttime = get_living_weather_data(DECODE_KEY,QUERY,DATE,lasttime,result)

In [64]:
result

{'getAirDiffusionIdx': '50',
 'getDiscomfortIdx': '77',
 'getHeatFeelingIdx': '24',
 'getSenTaIdx': '24',
 'getUVIdx': '6'}

# Public Air Dust Data Read

In [2]:
import urllib
from urllib.parse import urlencode, quote_plus, unquote
from urllib.request import urlopen
import requests
import json
import time

In [28]:

url = f"http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnStatsSvc/getMsrstnAcctoLastDcsnDnsty"

#-*- coding:utf-8 -*-
DECODE_KEY = unquote('cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D') 
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : DECODE_KEY, quote_plus('stationName') : '종로구', quote_plus('searchCondition') : 'DAILY' })
request = requests.get(url + queryParams)

request



<Response [200]>

In [19]:
request = requests.get('http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnStatsSvc/getDatePollutnStatInfo?serviceKey=cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D&numOfRows=10&pageNo=1&searchDataTime=2008-01&statArticleCondition=%EB%8F%84%EC%8B%9C%EB%8C%80%EA%B8%B0&')

In [29]:
request.text

'<?xml version="1.0" encoding="UTF-8"?>\r\n\r\n\r\n\r\n\r\n<response>\r\n\t<header>\r\n\t\t<resultCode>00</resultCode>\r\n\t\t<resultMsg>NORMAL SERVICE.</resultMsg>\r\n\t</header>\r\n\t<body>\r\n\t\t<items>\r\n\t\t\t\r\n\t\t\t\t<item>\r\n\t\t\t\t\t<dataTime>2006-01-01</dataTime>\r\n\t\t\t\t\t<so2Avg> 0.008</so2Avg>\r\n\t\t\t\t\t<coAvg> 0.8</coAvg>\r\n\t\t\t\t\t<o3Avg> 0.002</o3Avg>\r\n\t\t\t\t\t<no2Avg> 0.039</no2Avg>\r\n\t\t\t\t\t<pm10Avg>   59</pm10Avg>\r\n\t\t\t\t</item>\r\n\t\t\t\r\n\t\t\t\t<item>\r\n\t\t\t\t\t<dataTime>2006-01-02</dataTime>\r\n\t\t\t\t\t<so2Avg> 0.008</so2Avg>\r\n\t\t\t\t\t<coAvg> 0.7</coAvg>\r\n\t\t\t\t\t<o3Avg> 0.010</o3Avg>\r\n\t\t\t\t\t<no2Avg> 0.028</no2Avg>\r\n\t\t\t\t\t<pm10Avg>   65</pm10Avg>\r\n\t\t\t\t</item>\r\n\t\t\t\r\n\t\t\t\t<item>\r\n\t\t\t\t\t<dataTime>2006-01-03</dataTime>\r\n\t\t\t\t\t<so2Avg> 0.007</so2Avg>\r\n\t\t\t\t\t<coAvg> 0.5</coAvg>\r\n\t\t\t\t\t<o3Avg> 0.015</o3Avg>\r\n\t\t\t\t\t<no2Avg> 0.022</no2Avg>\r\n\t\t\t\t\t<pm10Avg>   30</pm10A